 ## HW4 - EuroSAT Land Use and Land Cover Classification using Deep Learning
 
In this homework your task is to implement deep learning models to solve a typical problem in satellite imaging  using a benchmark dataset. The homework was designed to make you work on increasingly more complex models. We hope that the homework will be very helpful to improve your skills and knowledge in deep learning!


### S1:

- Visit the EuroSAT data description page and download the data: https://github.com/phelber/eurosat

- Split the data into training (50%) and testing sets (50%), stratified on class labels (equal percentage of each class type in train and test sets).

- Convert each RGB image to grayscale and flatten the images into a data matrix (n x p: n = #samples, p = #pixels in each image)


In [1]:
import cv2
from PIL import Image
import PIL
import numpy as np
import pandas as pd
import os
from skimage import io


import sys
import numpy as np
import pandas as pd
import os
from skimage.transform import resize
import pickle
import matplotlib
import matplotlib.pyplot as plt
from sklearn import cluster

import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import StratifiedKFold

from sklearn.utils import shuffle

from sklearn import preprocessing


from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout,Flatten,Conv2D, MaxPooling2D
from keras.utils.np_utils import to_categorical
from tensorflow.keras.optimizers import RMSprop
import numpy as np

In [2]:
DIRECTORY = r'EuroSAT\2750'

CATEGORIES = ['SeaLake', 'River','Residential','PermanentCrop',
              'Pasture','Industrial','Highway','HerbaceousVegetation',
             'Forest','AnnualCrop']
num_class=(len(CATEGORIES))
image=PIL.Image.open('EuroSAT/2750/Forest/Forest_1.jpg',mode='r', formats=None)
pix=image.load()
print(image.size)
w=image.size[0]
h=image.size[1]
n_spectrum = 2


(64, 64)


In [4]:
data = []
X=[]
Y=[]
for category in CATEGORIES:
    path = os.path.join(DIRECTORY, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        label = CATEGORIES.index(category)
        arr = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        new_arr = cv2.resize(arr, (64, 64))
        X.append(new_arr)
        Y.append(label)
        data.append([new_arr, label])

In [5]:
X_value=np.array(X)
X_value=X_value.reshape(X_value.shape[0],-1)
X_value=X_value.astype('float32')
X_value/=255
Y_value=np.array(Y)


In [8]:

x_train, x_test, y_train, y_test = train_test_split(X_value, Y_value, stratify = Y_value, train_size = 0.5, random_state=42)
y_train = to_categorical(y_train, num_class)
y_test = to_categorical(y_test, num_class)

In [9]:
y_test

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

In [10]:

print(Y_value[(Y_value==2)].shape)
print(y_train[(y_train==2)].shape)
batch_size=(x_train.shape)
batch_size

(3000,)
(0,)


(13500, 4096)

### S2:

- Implement a first deep learning model (M.1) using a fully connected network with a single fully connected layer (i.e: input layer + fully connected layer as the output layer). 

###  CNN:
1. reduce para: kernel sharing para, proximity in pixel, kernel is learned by the DNN

Q2.1: Calculate classification accuracy on the test data.  

In [18]:
model_1 = Sequential()
model_1.add(Dense(num_class, activation='softmax', input_shape=(batch_size[1],)))
model_1.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 10)                40970     
                                                                 
Total params: 40,970
Trainable params: 40,970
Non-trainable params: 0
_________________________________________________________________


In [19]:
model_1.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

In [20]:
history = model_1.fit(x_train, y_train,
                    batch_size=64,
                      # unbiased sampling, but large the size the lower variance
                      # 32->64
                    epochs=50,
                    verbose=0,
                    validation_data=(x_test, y_test))

In [21]:
score = model_1.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 2.2163143157958984
Test accuracy: 0.21607407927513123


### S3:

- Implement a second deep learning model (M.2) adding an additional fully connected hidden layer (with an arbitrary number of nodes) to the previous model. 


Q3.1: Calculate classification accuracy on the test data.  

In [11]:
model_2 = Sequential()
model_2.add(Dense(512, activation='relu', input_shape=(batch_size[1],)))
model_2.add(Dense(num_class, activation='softmax'))
model_2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               2097664   
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 2,102,794
Trainable params: 2,102,794
Non-trainable params: 0
_________________________________________________________________


In [12]:
model_2.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

In [13]:
history = model_2.fit(x_train, y_train,
                    batch_size=64,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))

Epoch 1/10
211/211 [==============================] - 6s 27ms/step - loss: 2.8946 - accuracy: 0.1164 - val_loss: 2.3006 - val_accuracy: 0.0959
Epoch 2/10
211/211 [==============================] - 6s 26ms/step - loss: 2.2485 - accuracy: 0.1409 - val_loss: 2.1950 - val_accuracy: 0.1385
Epoch 3/10
211/211 [==============================] - 5s 26ms/step - loss: 2.1863 - accuracy: 0.1734 - val_loss: 2.1338 - val_accuracy: 0.2035
Epoch 4/10
211/211 [==============================] - 6s 26ms/step - loss: 2.1405 - accuracy: 0.2039 - val_loss: 2.1403 - val_accuracy: 0.1927
Epoch 5/10
211/211 [==============================] - 6s 26ms/step - loss: 2.0994 - accuracy: 0.2353 - val_loss: 2.1026 - val_accuracy: 0.2350
Epoch 6/10
211/211 [==============================] - 6s 26ms/step - loss: 2.0660 - accuracy: 0.2526 - val_loss: 2.0115 - val_accuracy: 0.3034
Epoch 7/10
211/211 [==============================] - 6s 26ms/step - loss: 2.0277 - accuracy: 0.2612 - val_loss: 1.9958 - val_accuracy: 0.2938

In [14]:
score = model_2.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.9432449340820312
Test accuracy: 0.298370361328125


### S4:

- Implement a third deep learning model (M.3) adding two additional fully connected hidden layers (with arbitrary number of nodes) for a total of four, as well as drop-out layers to the previous model. 

#todo: how many layers in total

Q4.1: Calculate classification accuracy on the test data.

Q4.2: Compare against previous models. Which model was the "best"? Why?

In [21]:
# softmax: multi-class prob
# relu: positive value 
model_3 = Sequential()
model_3.add(Dense(512, activation='relu', input_shape=(batch_size[1],)))
model_3.add(Dense(216, activation='relu'))
model_3.add(Dense(64, activation='relu'))
model_3.add(Dropout(0.25))
model_3.add(Dense(num_class, activation='softmax'))
model_3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 512)               2097664   
                                                                 
 dense_5 (Dense)             (None, 216)               110808    
                                                                 
 dense_6 (Dense)             (None, 64)                13888     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_7 (Dense)             (None, 64)                4160      
                                                                 
 dense_8 (Dense)             (None, 10)                650       
                                                                 
Total params: 2,227,170
Trainable params: 2,227,170
No

In [22]:
model_3.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

In [23]:
history = model_3.fit(x_train, y_train,
                    batch_size=64,
                    epochs=50,
                    verbose=1,
                    validation_data=(x_test, y_test))


Epoch 1/50
211/211 [==============================] - 7s 29ms/step - loss: 2.3488 - accuracy: 0.1087 - val_loss: 2.2527 - val_accuracy: 0.1130
Epoch 2/50
211/211 [==============================] - 6s 28ms/step - loss: 2.2322 - accuracy: 0.1532 - val_loss: 2.1751 - val_accuracy: 0.2103
Epoch 3/50
211/211 [==============================] - 6s 30ms/step - loss: 2.1444 - accuracy: 0.2046 - val_loss: 2.1112 - val_accuracy: 0.2281
Epoch 4/50
211/211 [==============================] - 6s 31ms/step - loss: 2.0348 - accuracy: 0.2415 - val_loss: 1.9602 - val_accuracy: 0.2501
Epoch 5/50
211/211 [==============================] - 6s 30ms/step - loss: 1.9403 - accuracy: 0.2691 - val_loss: 1.8702 - val_accuracy: 0.2826
Epoch 6/50
211/211 [==============================] - 6s 28ms/step - loss: 1.8797 - accuracy: 0.2918 - val_loss: 1.7940 - val_accuracy: 0.3278
Epoch 7/50
211/211 [==============================] - 6s 27ms/step - loss: 1.8368 - accuracy: 0.3084 - val_loss: 1.7474 - val_accuracy: 0.3409

In [24]:
score = model_3.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.773932695388794
Test accuracy: 0.3383703827857971


### S5:

- Take the original RGB images and do not vectorize them. Use these images as the data input for the following models (M.4 and M.5).
- Implement a fourth CNN model (M.4) that includes the following layers: Conv2D, MaxPooling2D, Dropout, Flatten, Dense. 

Q5.1: Calculate classification accuracy on the test data.

Q5.2: Compare against previous models. Which model was the "best"? Why?

In [33]:
data = []
X=[]
Y=[]
for category in CATEGORIES:
    path = os.path.join(DIRECTORY, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        label = CATEGORIES.index(category)
        arr = cv2.imread(img_path, cv2.IMREAD_COLOR)
        new_arr = cv2.resize(arr, (64, 64))
        X.append(new_arr)
        Y.append(label)
        data.append([new_arr, label])

In [34]:
X=np.array(X)
X.shape

(27000, 64, 64, 3)

In [52]:

X_value=X.astype('float32')
X_value/=255
Y_value=np.array(Y)

x_train, x_test, y_train, y_test = train_test_split(X_value, Y_value, stratify = Y_value, train_size = 0.5, random_state=42)
y_train = to_categorical(y_train, num_class)
y_test = to_categorical(y_test, num_class)

input_shape=(64,64,3)

In [53]:
model_4 = Sequential()

model_4.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',input_shape=input_shape
                ))

model_4.add(MaxPooling2D(pool_size=(2, 2)))
model_4.add(Dropout(0.25))
model_4.add(Flatten())
model_4.add(Dense(num_class, activation='softmax'))
model_4.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 31, 31, 32)       0         
 2D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 31, 31, 32)        0         
                                                                 
 flatten_2 (Flatten)         (None, 30752)             0         
                                                                 
 dense_3 (Dense)             (None, 10)                307530    
                                                                 
Total params: 308,426
Trainable params: 308,426
Non-trainable params: 0
________________________________________________

In [54]:
model_4.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

In [55]:
history = model_4.fit(x_train, y_train,
                    batch_size=64,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))

score = model_4.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/10
211/211 [==============================] - 16s 76ms/step - loss: 1.6132 - accuracy: 0.4103 - val_loss: 1.3055 - val_accuracy: 0.4965
Epoch 2/10
211/211 [==============================] - 16s 74ms/step - loss: 1.2435 - accuracy: 0.5479 - val_loss: 1.1634 - val_accuracy: 0.5776
Epoch 3/10
211/211 [==============================] - 16s 75ms/step - loss: 1.0984 - accuracy: 0.6079 - val_loss: 1.3685 - val_accuracy: 0.4753
Epoch 4/10
211/211 [==============================] - 16s 76ms/step - loss: 1.0173 - accuracy: 0.6431 - val_loss: 1.0135 - val_accuracy: 0.6250
Epoch 5/10
211/211 [==============================] - 16s 77ms/step - loss: 0.9674 - accuracy: 0.6659 - val_loss: 0.9851 - val_accuracy: 0.6285
Epoch 6/10
211/211 [==============================] - 16s 77ms/step - loss: 0.8872 - accuracy: 0.6925 - val_loss: 0.9586 - val_accuracy: 0.6673
Epoch 7/10
211/211 [==============================] - 16s 77ms/step - loss: 0.8563 - accuracy: 0.7022 - val_loss: 0.8896 - val_accuracy:

### S6:

- Using RGB images from S5, implement a fifth deep learning model (M.5) targeting accuracy that will outperform all previous models. You are free to use any tools and techniques, as well as pre-trained models for transfer learning. 

Q6.1: Describe the model you built, and why you chose it.

Q6.2: Calculate classification accuracy on the test data.

Q6.3: Compare against previous models. Which model was the "best"? Why?

Q6.4: What are the two classes with the highest labeling error? Explain using data and showing mis-classified examples.

In [4]:
model_5 = Sequential()
model_5.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model_5.add(MaxPooling2D((2, 2)))
model_5.add(Dropout(0.25))
model_5.add(Conv2D(64, (3, 3), activation='relu'))
model_5.add(MaxPooling2D((2, 2)))
model_5.add(Conv2D(64, (3, 3), activation='relu'))
model_5.add(Flatten())
model_5.add(Dense(num_class, activation='softmax'))
model_5.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 31, 31, 32)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 31, 31, 32)        0         
                                                                 
 conv2d_4 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 12, 12, 64)       

In [66]:
model_5.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model_5.fit(x_train, y_train,
                    batch_size=64,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))

score = model_5.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/10
211/211 [==============================] - 31s 146ms/step - loss: 1.6961 - accuracy: 0.3583 - val_loss: 1.4470 - val_accuracy: 0.4362
Epoch 2/10
211/211 [==============================] - 32s 150ms/step - loss: 1.2250 - accuracy: 0.5611 - val_loss: 1.0498 - val_accuracy: 0.5990
Epoch 3/10
211/211 [==============================] - 32s 152ms/step - loss: 1.0013 - accuracy: 0.6429 - val_loss: 1.2746 - val_accuracy: 0.5451
Epoch 4/10
211/211 [==============================] - 33s 155ms/step - loss: 0.8809 - accuracy: 0.6790 - val_loss: 0.8854 - val_accuracy: 0.6628
Epoch 5/10
211/211 [==============================] - 33s 155ms/step - loss: 0.7959 - accuracy: 0.7105 - val_loss: 0.7491 - val_accuracy: 0.7120
Epoch 6/10
211/211 [==============================] - 35s 166ms/step - loss: 0.7185 - accuracy: 0.7434 - val_loss: 0.7868 - val_accuracy: 0.7237
Epoch 7/10
211/211 [==============================] - 35s 164ms/step - loss: 0.6575 - accuracy: 0.7613 - val_loss: 0.7216 - val_ac

### S7:

- Apply your best model on multispectral images. You may use whichever image channels you wish, so long as you use more than just RGB (although you are not required to use any color channels).

Q7.1: Calculate classification accuracy on the test data.

Q7.2: Compare against results using RGB images.

In [48]:
DIRECTORY = r'EuroSATallBands\\ds\\images\\remote_sensing\\otherDatasets\\sentinel_2\\tif'

CATEGORIES = ['SeaLake', 'River','Residential','PermanentCrop',
              'Pasture','Industrial','Highway','HerbaceousVegetation',
             'Forest','AnnualCrop']
num_class=(len(CATEGORIES))
image=io.imread('Forest_1.tif')
image

array([[[1152,  830,  668, ..., 1592,  592, 3894],
        [1152,  830,  668, ..., 1592,  592, 3894],
        [1152,  851,  679, ..., 1607,  593, 3951],
        ...,
        [1153,  813,  588, ...,  828,  335, 2060],
        [1153,  811,  618, ...,  830,  344, 2044],
        [1153,  820,  615, ...,  844,  360, 2055]],

       [[1152,  830,  668, ..., 1592,  592, 3894],
        [1152,  830,  668, ..., 1592,  592, 3894],
        [1152,  851,  679, ..., 1607,  593, 3951],
        ...,
        [1153,  813,  588, ...,  828,  335, 2060],
        [1153,  811,  618, ...,  830,  344, 2044],
        [1153,  820,  615, ...,  844,  360, 2055]],

       [[1153,  839,  670, ..., 1593,  592, 3874],
        [1153,  839,  670, ..., 1593,  592, 3874],
        [1153,  846,  672, ..., 1604,  592, 3931],
        ...,
        [1154,  832,  608, ...,  851,  338, 2082],
        [1154,  832,  627, ...,  864,  352, 2088],
        [1154,  811,  619, ...,  887,  374, 2123]],

       ...,

       [[1149,  822,  62

In [5]:
data = []
X=[]
Y=[]
for category in CATEGORIES:
    path = os.path.join(DIRECTORY, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        label = CATEGORIES.index(category)
        arr = io.imread(img_path)
#         new_arr = cv2.resize(arr, (64, 64))
        X.append(arr)
        Y.append(label)
        data.append([arr, label])

In [7]:
X=np.array(X).astype('float32')
X.shape
X_value=X[:,:,:,:4]
X_value/=255
Y_value=np.array(Y)

x_train, x_test, y_train, y_test = train_test_split(X_value, Y_value, stratify = Y_value, train_size = 0.5, random_state=42)
y_train = to_categorical(y_train, num_class)
y_test = to_categorical(y_test, num_class)

In [13]:
model_6 = Sequential()
model_6.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 4)))
model_6.add(MaxPooling2D((2, 2)))
model_6.add(Dropout(0.25))
model_6.add(Conv2D(64, (3, 3), activation='relu'))
model_6.add(MaxPooling2D((2, 2)))
model_6.add(Conv2D(64, (3, 3), activation='relu'))
model_6.add(Flatten())
model_6.add(Dense(num_class, activation='softmax'))
model_6.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 62, 62, 32)        1184      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 31, 31, 32)       0         
 2D)                                                             
                                                                 
 dropout_3 (Dropout)         (None, 31, 31, 32)        0         
                                                                 
 conv2d_10 (Conv2D)          (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 conv2d_11 (Conv2D)          (None, 12, 12, 64)       

In [14]:
y_test.shape

(13500, 10)

In [15]:
model_6.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model_6.fit(x_train, y_train,
                    batch_size=64,
                    epochs=1,
                    verbose=1,
                    validation_data=(x_test, y_test))

score = model_6.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

ValueError: in user code:

    File "C:\Users\Xiaoyi WU\.conda\envs\py38\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Xiaoyi WU\.conda\envs\py38\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Xiaoyi WU\.conda\envs\py38\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Xiaoyi WU\.conda\envs\py38\lib\site-packages\keras\engine\training.py", line 859, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\Xiaoyi WU\.conda\envs\py38\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\Xiaoyi WU\.conda\envs\py38\lib\site-packages\keras\engine\input_spec.py", line 264, in assert_input_